In [1]:
from datasets import load_dataset
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from datasets import DatasetDict

## CUDA

In [ ]:
torch.cuda.empty_cache()

In [ ]:
torch.cuda.get_device_name(0)

## DatasetDict (関係ない)

In [ ]:
train_df = pd.DataFrame({
     "label" : [1, 2, 3],
     "text" : ["apple", "pear", "strawberry"]
})

test_df = pd.DataFrame({
     "label" : [2, 2, 1],
     "text" : ["banana", "pear", "apple"]
})

In [ ]:
train_dataset = Dataset.from_dict(train_df)
test_dataset = Dataset.from_dict(test_df)
my_dataset_dict = DatasetDict({'train':train_dataset, 'test':test_dataset})

In [ ]:
my_dataset_dict

In [ ]:
my_dataset_dict['train']['label']

In [ ]:
df = pd.DataFrame({"a": [1, 2, 3],'b':[4,5,6]})
dataset = Dataset.from_pandas(df)

In [ ]:
dataset

# wolfdata導入

In [2]:
wolfdata = pd.read_excel('Merged_1.xlsx', index_col=1)
wolfdata = wolfdata.drop(['Unnamed: 0_y','Unnamed: 0_x'],axis=1)
wolfdata = Dataset.from_pandas(wolfdata)

In [5]:
wolfsentence = wolfdata['対話ログ']

## 前処理

https://note.com/narudesu/n/na35de30a583a#JUDmI   前処理

In [3]:
!pip install demoji
import demoji
import re

--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.9/dist-packages/pip/_internal/utils/logging.py", line 134, in __rich_console__
    for line in lines:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/segment.py", line 245, in split_lines
    for segment in segments:
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/console.py", line 1368, in render
    renderable = rich_cast(renderable)
  File "/usr/local/lib/python3.9/dist-packages/pip/_vendor/rich/protocol.py", line 36

In [6]:
for i in range(len(wolfsentence)):
    wolfsentence[i] = wolfsentence[i].replace('\n','').replace('\r','')
    wolfsentence[i] = demoji.replace(string=wolfsentence[i], repl='')
    wolfsentence[i] = re.sub('[Σ└───┌┐│∩┘ヽ･ω･´ﾉ★▽†▼｜＿◆▼●□!"#$%&\'\\\\()*+,-./:;<=>?@[\\]^_`{|}~「」〔〕“”〈〉『』【】＆＊・（）＄＃＠。、？！｀＋￥％■]', '', wolfsentence[i])
    wolfsentence[i] = re.sub('[0-999]*\u3000', '', wolfsentence[i])
    wolfsentence[i] = re.sub('[0-999]*＞', '', wolfsentence[i])
    wolfsentence[i] = re.sub('[0-99999999999]', '', wolfsentence[i])

## 例 SentenceTransformerでVectorizeしてCosineSimilarity計算します

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [8]:
sentences = [
    'こいつは黒だ！',
    '他灰で気になったのはあんま見てないからアレですが、ペタ君は気になったですね。',         
    'なんか、リーザの事理解出来ないから占いたいからリーザの事理解出来ないから黒いになってませんかにゃん？',
    'ニコ兄が灰雑感落としているけど、白と黒ともとれる感じでかいてて「で、とりあえず今のとこ誰疑ってるの？」みたいなの感じたな。ちょっと狂っぽい印象。あとなぜかヤコ兄の雑感がなくて…ヤコ兄せつないなと思った。',
    'フリさんそんなに黒いか、と言えばそう思ってないのですが、いい代案が出てこないです…。',
    'こいつは偽物だ'
]
sentence_embeddings = model.encode(sentences)

In [9]:
wolfsentence_embeddings = model.encode(wolfsentence)

In [10]:
wolf_cos_2 = cosine_similarity(
    [sentence_embeddings[5]],
    wolfsentence_embeddings
)

In [11]:
wolfsentence[np.argmax(wolf_cos_2)]

'Σぎゃぼっナイスつっこみニコラスさんいやあ私ってホントバカですねぇ｡pqｸｽｯ'

# MeCabTokenizer BertJapanese

In [ ]:
!pip install fugashi
import fugashi

In [ ]:
!pip install transformers["ja"]

In [ ]:
from transformers import AutoModel, AutoTokenizer
bertjapanese = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese")
tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese")

In [ ]:
line = sentences[4]
inputs = tokenizer(line, return_tensors="pt")

In [ ]:
print(tokenizer.decode(inputs["input_ids"][0]))